<a href="https://colab.research.google.com/github/karthikmohan1702/EVA6/blob/main/EVA6_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function # forces the use of print statements without parenthesis
import torch # It is a pytorch library
import torch.nn as nn #In PyTorch we define Models as subclasses of torch.nn.Module
import torch.nn.functional as F # torch.nn.Functional contains convolution operation
import torch.optim as optim # contains various optimization algorithms like adam, SGD
from torchvision import datasets, transforms # consists of datasets (like CIFAR, COCO, MNIST), & image transformations for computer vision.

In [2]:
# Here we define the architecture of our model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) # 28x28x1 | (3x3x1)x16 | 28x28x16, RF = 3
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # 28x28x16 | (3x3x16)x32 | 28x28x32, RF = 5
        self.pool1 = nn.MaxPool2d(2, 2) # 28x28x32 | 2x2 | 14x14x32, RF = 10
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1) # 14x14x32 | (3x3x32)x64 | 14x14x64, RF = 12
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1) # 14x14x64 | (3x3x64)x128 | 14x14x128, RF = 14 
        self.pool2 = nn.MaxPool2d(2, 2) # 14x14x128 | 2x2 | 7x7x128, RF = 28
        self.conv5 = nn.Conv2d(128, 256, 3) # 7x7x128 | (3x3x128)x256 | 5x5x256, RF = 30
        self.conv6 = nn.Conv2d(256, 512, 3) # 5x5x256 | (3x3x256)x512 | 3x3x512, RF = 32
        self.conv7 = nn.Conv2d(512, 10, 3) # 3x3x512 | (3x3x512)x10 | 1x1x10, RF = 34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
# Initializing the device & looking at the summary of the mode, like how the 
# parameters are populated in the each layer that we have created
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
            Conv2d-2           [-1, 32, 28, 28]           4,640
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
            Conv2d-5          [-1, 128, 14, 14]          73,856
         MaxPool2d-6            [-1, 128, 7, 7]               0
            Conv2d-7            [-1, 256, 5, 5]         295,168
            Conv2d-8            [-1, 512, 3, 3]       1,180,160
            Conv2d-9             [-1, 10, 1, 1]          46,090
Total params: 1,618,570
Trainable params: 1,618,570
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.75
Params size (MB): 6.17
Estimated Total Size (MB): 6.93
---------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
# Here we are loading the dataset & applying transformations on them & 
# normalizing it. Also intializing the batch size.
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
# here we have functions for train & test which tells how the data is going
# to be passed in batches & the type of optimizer and loss that's implemented
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
# We train the model on number of epochs mentioned
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.10218050330877304 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.53it/s]



Test set: Average loss: 0.0706, Accuracy: 9772/10000 (98%)

